In [ ]:
-- -- Create database and schema
-- CREATE OR REPLACE DATABASE frostview_test;
-- CREATE OR REPLACE SCHEMA frostview_test.public;

-- -- Create table
-- CREATE OR REPLACE TABLE frostview_test.public.sample_table (
--     id INT NOT NULL,
--     email STRING,
--     age INT,
--     city STRING NOT NULL
-- );

-- -- Insert sample data
-- INSERT INTO frostview_test.public.sample_table (id, email, age, city) VALUES
--     (1, 'a@example.com', 30, 'New York'),
--     (2, 'b@example.com', 25, 'London'),
--     (3, 'c@example.com', NULL, 'Tel Aviv'),
--     (4, NULL, 40, 'Paris'),
--     (5, NULL, NULL, 'Tokyo');


In [ ]:
CREATE DATABASE IF NOT EXISTS FROSTVIEW;
CREATE SCHEMA IF NOT EXISTS FROSTVIEW.SYSTEM_TABLES;

In [ ]:
CREATE TABLE IF NOT EXISTS frostview.system_tables.frostview_log (
    run_id STRING,
    run_time TIMESTAMP,
    database_name STRING,
    schema_name STRING,
    table_name STRING,
    column_name STRING,
    test_type STRING, -- 'not_null' or 'unique'
    passed BOOLEAN,
    num_failed INT,
    error_message STRING
);


In [ ]:
CREATE TABLE IF NOT EXISTS frostview.system_tables.test_config (
    database_name STRING,
    schema_name STRING,
    table_name STRING,
    column_name STRING,
    test_type STRING,             -- 'not_null' or 'unique'
    schedule_enabled BOOLEAN,     -- TRUE if scheduled
    schedule_cron STRING,         -- Cron expression, e.g. '24h'
    triggered_by STRING,          -- For future use: e.g. 'task:data_ingest_x'
    created_at TIMESTAMP,
    updated_at TIMESTAMP
);


In [ ]:
import streamlit as st
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import col
import uuid
from datetime import datetime
import pandas as pd

In [ ]:
# Connect to Snowflake
session = Session.builder.configs({
    "connection_name": "my_connection"
}).getOrCreate()

In [ ]:
st.title("FrostView: Table Validator")

with st.form("table_form"):
    user_input = st.text_input("Enter full table name (format: DB.SCHEMA.TABLE):")
    submitted = st.form_submit_button("Submit")

if submitted:
    parts = user_input.strip().split(".")
    if len(parts) != 3:
        st.error("Please enter the table name in format: DB.SCHEMA.TABLE")
    else:
        try:
            session.sql(f"SELECT * FROM {user_input.strip()} LIMIT 1").collect()
            st.success(f"Table '{user_input.strip()}' exists and is accessible.")
        except Exception as e:
            st.error(f"Table access failed: {e}")


In [ ]:
def generate_run_id():
    return f"fv_run_{uuid.uuid4().hex[:8]}"

In [ ]:
def run_not_null_test(session, db, schema, table, column):
    run_id = generate_run_id()
    run_time = datetime.now()
    full_table = f"{db}.{schema}.{table}"

    try:
        rows = session.sql(f"""
            SELECT COUNT(*) AS num_failed
            FROM {full_table}
            WHERE {column} IS NULL
            LIMIT 1
        """).collect()

        result = rows[0]["NUM_FAILED"] if rows else 0
        passed = result == 0
        error_message = None

    except Exception as e:
        result = None
        passed = False
        error_message = str(e)

    log_df = pd.DataFrame([{
        "run_id": run_id,
        "run_time": run_time,
        "database_name": db,
        "schema_name": schema,
        "table_name": table,
        "column_name": column,
        "test_type": "not_null",
        "passed": passed,
        "num_failed": result,
        "error_message": error_message
    }])

    session.sql("USE DATABASE frostview").collect()
    session.sql("USE SCHEMA system_tables").collect()
    log_table = "FROSTVIEW_LOG"
    log_df.columns = [c.upper() for c in log_df.columns]
    session.write_pandas(
        log_df,
        table_name=log_table,
        auto_create_table=False,
        overwrite=False
    )

    return passed, log_df


In [ ]:
def run_unique_test(session, db, schema, table, column):
    run_id = generate_run_id()
    run_time = datetime.now()
    full_table = f"{db}.{schema}.{table}"

    try:
        rows = session.sql(f"""
            SELECT COUNT(*) AS num_failed FROM (
                SELECT {column}, COUNT(*) AS cnt
                FROM {full_table}
                GROUP BY {column}
                HAVING COUNT(*) > 1
            )
            LIMIT 1
        """).collect()

        result = rows[0]["NUM_FAILED"] if rows else 0
        passed = result == 0
        error_message = None

    except Exception as e:
        result = None
        passed = False
        error_message = str(e)

    log_df = pd.DataFrame([{
        "run_id": run_id,
        "run_time": run_time,
        "database_name": db,
        "schema_name": schema,
        "table_name": table,
        "column_name": column,
        "test_type": "unique",
        "passed": passed,
        "num_failed": result,
        "error_message": error_message
    }])

    session.sql("USE DATABASE frostview").collect()
    session.sql("USE SCHEMA system_tables").collect()
    log_table ="FROSTVIEW_LOG"
    log_df.columns = [c.upper() for c in log_df.columns]
    session.write_pandas(
        log_df,
        table_name=log_table,
        auto_create_table=False,
        overwrite=False
    )

    return passed, log_df

In [ ]:
columns = session.table(user_input.strip()).columns
db, schema, table = user_input.strip().split(".")

In [ ]:
def get_active_tests(session, db='frostview', 
                     schema='system_tables', table='frost_config'):
    query = f"""
        SELECT column_name, test_type
        FROM frostview.system_tables.test_config
        WHERE database_name = '{db}'
          AND schema_name = '{schema}'
          AND table_name = '{table}'
    """
    rows = session.sql(query).collect()

    # Build dict: { column: {test_type: True/False} }
    config = {}
    for row in rows:
        col = row["COLUMN_NAME"]
        test = row["TEST_TYPE"]
        config.setdefault(col, {})[test] = True

    return config


In [ ]:
active_config = get_active_tests(session)

In [ ]:
active_config

In [ ]:
def add_test_to_config(session, db, schema, table, column, test_type):
    now = datetime.now()
    session.sql(f"""
        INSERT INTO frostview.system_tables.test_config (
            database_name, schema_name, table_name,
            column_name, test_type,
            schedule_enabled, schedule_cron,
            triggered_by, created_at, updated_at
        )
        VALUES (
            '{db}', '{schema}', '{table}',
            '{column}', '{test_type}',
            TRUE, '24h', NULL,
            '{now}', '{now}'
        )
    """).collect()

In [ ]:
def remove_test_from_config(session, db, schema, table, column, test_type):
    session.sql(f"""
        DELETE FROM frostview.system_tables.test_config
        WHERE database_name = '{db}'
          AND schema_name = '{schema}'
          AND table_name = '{table}'
          AND column_name = '{column}'
          AND test_type = '{test_type}'
    """).collect()

In [ ]:
for col in columns:
    with st.expander(f"🧩 `{col}`", expanded=False):
        col_config = active_config.get(col, {})
        is_nn_scheduled = col_config.get("not_null", False)
        is_uq_scheduled = col_config.get("unique", False)

        # --- Ad-hoc run ---
        st.write("### ▶️ Run Once (Ad-hoc)")
        if st.button("Run NOT NULL now", key=f"run_nn_{col}"):
            passed, log = run_not_null_test(session, db, schema, table, col)
            st.success("✅ Passed" if passed else "❌ Failed")
            st.dataframe(log)

        if st.button("Run UNIQUE now", key=f"run_uq_{col}"):
            passed, log = run_unique_test(session, db, schema, table, col)
            st.success("✅ Passed" if passed else "❌ Failed")
            st.dataframe(log)

        # --- Scheduling UI ---
        st.write("### ⏰ Schedule Every 24 Hours")

        if not is_nn_scheduled:
            if st.button("📅 Schedule NOT NULL test", key=f"sched_nn_{col}"):
                add_test_to_config(session, db, schema, table, col, "not_null")
                st.success("Scheduled NOT NULL test for 24h")
        else:
            if st.button("❌ Unschedule NOT NULL", key=f"unsched_nn_{col}"):
                remove_test_from_config(session, db, schema, table, col, "not_null")
                st.warning("Unscheduled NOT NULL test")

        if not is_uq_scheduled:
            if st.button("📅 Schedule UNIQUE test", key=f"sched_uq_{col}"):
                add_test_to_config(session, db, schema, table, col, "unique")
                st.success("Scheduled UNIQUE test for 24h")
        else:
            if st.button("❌ Unschedule UNIQUE", key=f"unsched_uq_{col}"):
                remove_test_from_config(session, db, schema, table, col, "unique")
                st.warning("Unscheduled UNIQUE test")


In [ ]:
for col in columns:
    st.markdown(f"---\n#### Column: `{col}`")

    col_config = active_config.get(col, {})
    is_nn_scheduled = col_config.get("not_null", False)
    is_uq_scheduled = col_config.get("unique", False)

    col1, col2, col3 = st.columns(3)

    with col1:
        st.markdown("**NOT NULL test**")

        if st.button("▶️ Run now", key=f"run_nn_{col}"):
            passed, log = run_not_null_test(session, db, schema, table, col)
            st.success("✅ Passed" if passed else "❌ Failed")
            st.dataframe(log)

        if not is_nn_scheduled:
            if st.button("📅 Schedule 24h", key=f"sched_nn_{col}"):
                add_test_to_config(session, db, schema, table, col, "not_null")
                st.success("Scheduled every 24h")
        else:
            if st.button("❌ Unschedule", key=f"unsched_nn_{col}"):
                remove_test_from_config(session, db, schema, table, col, "not_null")
                st.warning("Unscheduled")

    with col2:
        st.markdown("**UNIQUE test**")

        if st.button("▶️ Run now", key=f"run_uq_{col}"):
            passed, log = run_unique_test(session, db, schema, table, col)
            st.success("✅ Passed" if passed else "❌ Failed")
            st.dataframe(log)

        if not is_uq_scheduled:
            if st.button("📅 Schedule 24h", key=f"sched_uq_{col}"):
                add_test_to_config(session, db, schema, table, col, "unique")
                st.success("Scheduled every 24h")
        else:
            if st.button("❌ Unschedule", key=f"unsched_uq_{col}"):
                remove_test_from_config(session, db, schema, table, col, "unique")
                st.warning("Unscheduled")

    with col3:
        st.markdown("**Status**")
        st.markdown(f"NOT NULL: {'✅' if is_nn_scheduled else '❌'}")
        st.markdown(f"UNIQUE: {'✅' if is_uq_scheduled else '❌'}")


In [ ]:
# # Parse table name from user input
# db, schema, table = user_input.strip().split(".")

# # Get column names from the table
# table_ref = session.table(user_input.strip())
# columns = table_ref.columns

# results = []

# st.markdown("### 🧪 Running Tests on All Columns")

# for col in columns:
#     st.write(f"🔍 Column: `{col}`")

#     # Run NOT NULL test
#     passed_nn, log_nn = run_not_null_test(session, db, schema, table, col)
#     st.write(f"  NOT NULL: {'✅ Pass' if passed_nn else '❌ Fail'}")
#     results.append(log_nn)

#     # Run UNIQUE test
#     passed_uniq, log_uniq = run_unique_test(session, db, schema, table, col)
#     st.write(f"  UNIQUE: {'✅ Pass' if passed_uniq else '❌ Fail'}")
#     results.append(log_uniq)

# # Show logs as a dataframe
# import pandas as pd
# st.markdown("### ✅ Test Logs Summary")
# st.dataframe(pd.concat(results, ignore_index=True))
